Задание

Руководитель отдела сопровождения клиентов хочет получить информацию о том, насколько хорошо сотрудники его отдела (аккаунт-менеджеры) справляются с одной из своих основных задач – пролонгацией договоров с клиентами. От аналитика он хочет получить отчет о пролонгациях сотрудников за 2023 год. 

В компание используется два коэффициента пролонгации:
Для проектов пролонгированных в первый месяц – отношении суммы отгрузки проектов пролонгированных в первый месяц после завершения к сумме отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов.
Для проектов, пролонгированных во второй месяц – отношение суммы отгрузки проектов, пролонгированных во второй месяц к сумме отгрузки последнего месяца проектов, не пролонгированных в первый. 

То есть, если нам нужно понять, насколько хорошо менеджер пролонгировал в мае, необходимо посчитать:
Сумму отгрузки проектов, завершившихся в апреле (за апрель) и сумму отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май). Коэффициент – отношение второй суммы к первой. 
Сумму проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март) и сумму отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле но есть в мае (за май). Коэффициент – отношение второй суммы к первой. 

Имеются два набора данных:

prolongations.csv
	id – id проекта
	month – последний месяц реализации проекта
	AM – ФИО ответственного аккаунт-менеджера (данные первичны по отношению к financial_data)

financial_data.csv:
	id – id проекта
	Причина дубля – причина, почему строки с одним и тем же id встречаются несколько раз
	Колонки с названием месяца – сумма отгрузки проекта в данный месяц
	Account – ФИО ответственного аккаунт-менеджера

Необходимо: 
Рассчитать коэффициенты пролонгации для каждого менеджера и для всего отдела в целом
за каждый месяц
за год
Сформировать аналитический отчет в гугл-таблицах или excel, на основании которого руководитель отдела будет принимать управленческие решения (обязательно присутствие коэффициентов пролонгации, в остальном отчет можно дополнить на свое усмотрение – дополнительные метрики и визуализация приветствуются).
Отчет должен быть читабельным и понятным для руководителя отдела.

В результате ждем 2 файла:
Код с подробными пояснениями логики расчетов в Google Colab или Jupiter Notebook
Аналитический отчет для руководителя отдела (в excel/google sheets)


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore') 

In [3]:
try:
    file_id_1 = '1r3grFMz_k-XuWhRNLOAZCoLDdVYg2C7X'
    file_id_2 = '1Nm25BZ_AKUksl3xJmvh_6DF3d_p6jYqf'
    url_1 = f'https://drive.google.com/uc?export=download&id={file_id}'
    url_2 = f'https://drive.google.com/uc?export=download&id={file_id}'
    financial_data = pd.read_csv(url_1)
    prolongations = pd.read_csv(url_2)
except:
    financial_data = pd.read_csv(r'C:\Users\maria\Downloads\financial_data.csv')
    prolongations = pd.read_csv(r'C:\Users\maria\Downloads\prolongations.csv')

In [4]:
financial_data

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,281,основные работы,"76 550,00","79 950,00","66 900,00","89 150,00","108 450,00","77 100,00","78 800,00","126 740,00","117 730,00","115 860,00","160 770,00","142 490,00","99 125,00","74 350,00","105 775,00","92 065,00",Соколова Анастасия Викторовна
447,281,доп работы,"21 450,00","13 300,00","15 900,00","19 850,00","17 350,00","14 650,00","15 900,00","3 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Соколова Анастасия Викторовна
448,785,NaN,NaN,NaN,NaN,"5 306,60","12 898,10","5 287,00","10 180,00","8 600,00","3 860,00","8 600,00","700,00","700,00",в ноль,в ноль,NaN,NaN,Соколова Анастасия Викторовна
449,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"58 200,00","58 200,00","58 200,00","58 200,00","58 200,00","58 200,00",NaN,NaN,Смирнова Ольга Владимировна


In [5]:
prolongations

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна
...,...,...,...
472,955,декабрь 2023,Смирнова Ольга Владимировна
473,1004,декабрь 2023,без А/М
474,281,декабрь 2023,Соколова Анастасия Викторовна
475,785,декабрь 2023,Соколова Анастасия Викторовна


Функция для обработки значений 'в ноль', 'стоп', 'end'

In [6]:
def process_special_values(value):
    """Обработка специальных значений: 'в ноль', 'стоп', 'end'"""
    if pd.isna(value):
        return np.nan
    
    # Если значение уже числовое
    if isinstance(value, (int, float)):
        return value
    
    # Обработка строковых значений
    if isinstance(value, str):
        value_str = str(value).strip()
        
        # Специальные значения
        if 'в ноль' in value_str.lower():
            return 0
        elif 'стоп' in value_str.lower() or 'end' in value_str.lower():
            return 'STOP'
        else:
            # Обработка числовых значений с запятыми как разделителями тысяч/десятичных
            # Убираем пробелы (разделители тысяч) и заменяем запятую на точку (десятичный разделитель)
            value_clean = value_str.replace(u'\xa0', '').replace(',', '.')
            
            # Проверяем, является ли результат числом
            try:
                # Пробуем преобразовать в float
                numeric_value = float(value_clean)
                return numeric_value
            except ValueError:
                # Если не получается, проверяем другие форматы
                # Может быть пустая строка или другие специальные значения
                if value_clean == '' or value_clean.lower() in ['nan', 'none', 'null']:
                    return np.nan
                else:
                    # Если это что-то непонятное, оставляем как есть для отладки
                    print(f"Нераспознанное значение: '{value}' -> '{value_clean}'")
                    return value_str
    
    # Для любых других типов возвращаем как есть
    return value

In [7]:
# Создаем копию данных для безопасной обработки
financial_clean = financial_data.copy()

# Список месяцев для анализа (из колонок financial_data)
month_columns = [col for col in financial_data.columns if col not in ['id', 'Причина дубля', 'Account']]
print(f"Месяцы для анализа: {month_columns}")

# Применяем обработку ко всем месячным колонкам
for col in month_columns:
    financial_clean[col] = financial_clean[col].apply(process_special_values)

Месяцы для анализа: ['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']


In [9]:
# Проверка внесенных изменений
financial_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             451 non-null    int64  
 1   Причина дубля  301 non-null    object 
 2   Ноябрь 2022    156 non-null    object 
 3   Декабрь 2022   159 non-null    object 
 4   Январь 2023    139 non-null    float64
 5   Февраль 2023   145 non-null    object 
 6   Март 2023      168 non-null    object 
 7   Апрель 2023    174 non-null    object 
 8   Май 2023       190 non-null    object 
 9   Июнь 2023      190 non-null    object 
 10  Июль 2023      195 non-null    object 
 11  Август 2023    199 non-null    object 
 12  Сентябрь 2023  186 non-null    object 
 13  Октябрь 2023   182 non-null    object 
 14  Ноябрь 2023    171 non-null    object 
 15  Декабрь 2023   146 non-null    float64
 16  Январь 2024    95 non-null     object 
 17  Февраль 2024   101 non-null    object 
 18  Account   

In [10]:
# Сохранение датафрейм в CSV-файл
financial_clean.to_csv('financial_clean.csv', index=False)

In [11]:
# Преобразование месяцев в datetime
month_columns = ['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 
                'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 
                'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
                'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']

# Создаем словарь для преобразования месяцев
month_mapping = {
    'ноябрь 2022': 'Ноябрь 2022', 'декабрь 2022': 'Декабрь 2022',
    'январь 2023': 'Январь 2023', 'февраль 2023': 'Февраль 2023',
    'март 2023': 'Март 2023', 'апрель 2023': 'Апрель 2023',
    'май 2023': 'Май 2023', 'июнь 2023': 'Июнь 2023',
    'июль 2023': 'Июль 2023', 'август 2023': 'Август 2023',
    'сентябрь 2023': 'Сентябрь 2023', 'октябрь 2023': 'Октябрь 2023',
    'ноябрь 2023': 'Ноябрь 2023', 'декабрь 2023': 'Декабрь 2023'
}

In [12]:
prolongations['month_norm'] = prolongations['month'].str.lower().map(month_mapping)

In [13]:
# Функция для определения активных проектов
def get_active_projects(month):
    """Возвращает список активных проектов в указанном месяце"""
    active_projects = []
    
    for idx, row in financial_clean.iterrows():
        project_id = row['id']
        account = row['Account']
        
        # Находим последний месяц с данными
        last_active_month = None
        for m in month_columns:
            if pd.notna(row[m]) and row[m] != 'STOP' and row[m] != 'end':
                last_active_month = m
            elif row[m] == 'STOP' or row[m] == 'end':
                break
        
        # Проект активен, если месяц находится до последнего активного месяца
        if last_active_month:
            month_index = month_columns.index(month)
            last_active_index = month_columns.index(last_active_month)
            if month_index <= last_active_index:
                active_projects.append((project_id, account))
    
    return active_projects

In [14]:
# Функция для расчета коэффициентов пролонгации
def calculate_prolongation_coefficients():
    results = {}
    managers = financial_clean['Account'].unique()
    
    for i in range(1, len(month_columns)):
        current_month = month_columns[i]
        previous_month = month_columns[i-1]
        
        #print(f"Расчет для {current_month}...")
        
        # Активные проекты в предыдущем месяце
        active_prev_month = get_active_projects(previous_month)
        active_projects_prev = set([p[0] for p in active_prev_month])
        
        # Пролонгированные проекты в текущем месяце
        prolonged_projects = prolongations[
            prolongations['month_norm'] == current_month
        ]['id'].unique()
        
        # Расчет по менеджерам
        manager_coefficients = {}
        for manager in managers:
            # Активные проекты менеджера в предыдущем месяце
            manager_active = [p[0] for p in active_prev_month if p[1] == manager]
            
            # Пролонгированные проекты менеджера
            manager_prolonged = []
            for project_id in prolonged_projects:
                project_manager = prolongations[
                    (prolongations['id'] == project_id) & 
                    (prolongations['month_norm'] == current_month)
                ]['AM'].iloc[0] if not prolongations[
                    (prolongations['id'] == project_id) & 
                    (prolongations['month_norm'] == current_month)
                ].empty else None
                
                if project_manager == manager:
                    manager_prolonged.append(project_id)
            
            # Расчет коэффициента
            if manager_active:
                coefficient = len(manager_prolonged) / len(manager_active)
            else:
                coefficient = 0
                
            manager_coefficients[manager] = coefficient
        
        # Общий коэффициент по отделу
        if active_projects_prev:
            total_coefficient = len(prolonged_projects) / len(active_projects_prev)
        else:
            total_coefficient = 0
            
        results[current_month] = {
            'manager_coefficients': manager_coefficients,
            'total_coefficient': total_coefficient,
            'active_projects_prev': len(active_projects_prev),
            'prolonged_projects': len(prolonged_projects)
        }
    
    return results

In [15]:
# Выполняем расчет
coefficients = calculate_prolongation_coefficients()

# Формируем итоговую таблицу
months = [month for month in coefficients.keys()]
managers = financial_clean['Account'].unique()

result_table = pd.DataFrame(index=managers, columns=months)

for month, data in coefficients.items():
    for manager, coeff in data['manager_coefficients'].items():
        result_table.loc[manager, month] = round(coeff, 4)

# Добавляем общий коэффициент по отделу
total_row = {}
for month, data in coefficients.items():
    total_row[month] = round(data['total_coefficient'], 4)

result_table.loc['ВСЕГО ПО ОТДЕЛУ'] = total_row

# Годовой коэффициент (среднее за все месяцы)
yearly_coefficients = {}
for manager in managers:
    manager_coeffs = [result_table.loc[manager, month] for month in months if pd.notna(result_table.loc[manager, month])]
    yearly_coefficients[manager] = round(np.mean(manager_coeffs), 4) if manager_coeffs else 0

# Годовой коэффициент по отделу
total_coeffs = [coefficients[month]['total_coefficient'] for month in months]
yearly_coefficients['ВСЕГО ПО ОТДЕЛУ'] = round(np.mean(total_coeffs), 4)

# Добавляем годовую строку
result_table['ГОДОВОЙ КОЭФФИЦИЕНТ'] = yearly_coefficients

print("РАСЧЕТ КОЭФФИЦИЕНТОВ ПРОЛОНГАЦИИ")
print("=" * 80)
print(result_table)

# Дополнительная статистика
print("\nДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА:")
print("=" * 50)
for month, data in coefficients.items():
    print(f"{month}:")
    print(f"  Активных проектов в предыдущем месяце: {data['active_projects_prev']}")
    print(f"  Пролонгированных проектов: {data['prolonged_projects']}")
    print(f"  Общий коэффициент: {data['total_coefficient']:.4f}")
    print()

РАСЧЕТ КОЭФФИЦИЕНТОВ ПРОЛОНГАЦИИ
                              Декабрь 2022 Январь 2023 Февраль 2023 Март 2023  \
Васильев Артем Александрович        0.2157      0.0581       0.1098    0.1358   
Соколова Анастасия Викторовна       0.1111       0.075       0.0811    0.0606   
Михайлов Андрей Сергеевич           0.5882      0.1818       0.1111    0.2222   
Попова Екатерина Николаевна         0.0714      0.0482       0.0886    0.0986   
Иванова Мария Сергеевна             0.3636         0.0       0.1429    0.1481   
Смирнова Ольга Владимировна         0.0405      0.0135          0.0     0.029   
Кузнецов Михаил Иванович            0.0345       0.069       0.0345    0.0357   
без А/М                                0.0         0.0          0.0       0.0   
Петрова Анна Дмитриевна                0.0         0.0          0.0       0.0   
Федорова Марина Васильевна             0.0         0.0          0.0       0.0   
ВСЕГО ПО ОТДЕЛУ                     0.2241      0.0725       0.1107     0.12

In [16]:
result_table

,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,ГОДОВОЙ КОЭФФИЦИЕНТ
Васильев Артем Александрович,0.2157,0.0581,0.1098,0.1358,0.1447,0.0735,0.1515,0.082,0.1455,0.1277,0.186,0.2,0.5862,0.0,0.0,0.1478
Соколова Анастасия Викторовна,0.1111,0.075,0.0811,0.0606,0.0615,0.0484,0.1356,0.0714,0.1224,0.2273,0.0811,0.2812,0.3462,0.0,0.0,0.1135
Михайлов Андрей Сергеевич,0.5882,0.1818,0.1111,0.2222,0.375,0.125,0.125,0.0,0.1667,0.8,0.0,0.5,0.5,0.0,0.0,0.2463
Попова Екатерина Николаевна,0.0714,0.0482,0.0886,0.0986,0.1159,0.0469,0.0645,0.0678,0.0741,0.1731,0.0,0.093,0.3023,0.0,0.0,0.0830
Иванова Мария Сергеевна,0.3636,0.0,0.1429,0.1481,0.0385,0.2917,0.25,0.3125,0.2727,0.1429,0.0,0.0,0.0,0.0,0.0,0.1309
Смирнова Ольга Владимировна,0.0405,0.0135,0.0,0.029,0.0,0.0435,0.0299,0.0615,0.0625,0.1667,0.1818,0.2667,0.439,0.0,0.0,0.0890
Кузнецов Михаил Иванович,0.0345,0.069,0.0345,0.0357,0.0,0.0,0.0,0.0,0.0,0.0417,0.125,0.25,0.4,0.0,0.0,0.0660
без А/М,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.1333
Петрова Анна Дмитриевна,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0667
Федорова Марина Васильевна,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0667


In [17]:
result_table_reset = result_table.reset_index()

In [18]:
# Сохраните датафрейм в CSV-файл
result_table_reset.to_csv('result_table_reset.csv', index=False)

Создание файла с дополнительной информацией

In [19]:
# Создаем данные
data = {
    'Месяц': ['Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 
              'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 
              'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024'],
    'Активных_проектов_в_предыдущем_месяце': [299, 276, 253, 244, 236, 223, 218, 201, 185, 171, 155, 134, 121, 109, 71],
    'Пролонгированных_проектов': [67, 20, 28, 31, 27, 22, 31, 23, 26, 41, 24, 40, 69, 0, 0],
    'Общий_коэффициент': [0.2241, 0.0725, 0.1107, 0.1270, 0.1144, 0.0987, 0.1422, 0.1144, 0.1405, 
                         0.2398, 0.1548, 0.2985, 0.5702, 0.0000, 0.0000]
}

# Создаем датафрейм
df = pd.DataFrame(data)

# Показываем датафрейм
print("Датафрейм:")
print(df)
print("\n" + "="*50 + "\n")

# Сохраняем в CSV файл
df.to_csv('проекты_аналитика.csv', index=False, encoding='utf-8-sig')

print("Файл 'проекты_аналитика.csv' успешно создан!")
print("\nИнформация о датафрейме:")
print(df.info())

Датафрейм:
            Месяц  Активных_проектов_в_предыдущем_месяце  \
0    Декабрь 2022                                    299   
1     Январь 2023                                    276   
2    Февраль 2023                                    253   
3       Март 2023                                    244   
4     Апрель 2023                                    236   
5        Май 2023                                    223   
6       Июнь 2023                                    218   
7       Июль 2023                                    201   
8     Август 2023                                    185   
9   Сентябрь 2023                                    171   
10   Октябрь 2023                                    155   
11    Ноябрь 2023                                    134   
12   Декабрь 2023                                    121   
13    Январь 2024                                    109   
14   Февраль 2024                                     71   

    Пролонгированных_проекто

После создания файла переходим в файл Google Sheets